## Configure imports and eager execution

In [2]:
from __future__ import absolute_import, division, print_function

import os


import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.11.0
Eager execution: True


## Download the dataset

In [3]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

8192/2194 [================================================================================================================] - 0s 0us/step
Local copy of the dataset file: C:\Users\OPTIPLEX\.keras\datasets\iris_training.csv


## Inspect the data
This dataset, iris_training.csv, is a plain text file that stores tabular data formatted as comma-separated values (CSV). Use the head -n5 command to take a peak at the first five entries:

In [6]:
# column order in CSV file
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


## classifiers

In [7]:
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

## creating a Tensorflow dataset

In [8]:
batch_size = 32

train_dataset = tf.contrib.data.make_csv_dataset(
    train_dataset_fp,
    batch_size, 
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

Each label is associated with string name (for example, "setosa"), but machine learning typically relies on numeric values. The label numbers are mapped to a named representation, such as:

0: Iris setosa
1: Iris versicolor
2: Iris virginica

In [9]:
features, labels = next(iter(train_dataset))

features

OrderedDict([('sepal_length',
              <tf.Tensor: id=60, shape=(32,), dtype=float32, numpy=
              array([7.2, 5. , 4.6, 5. , 7. , 5.4, 4.9, 5. , 6.9, 6.5, 5.2, 5. , 7.7,
                     5.1, 6.1, 7.7, 5.6, 7.4, 5.8, 6. , 4.6, 6.5, 6. , 5.4, 6.4, 5.6,
                     6. , 7.7, 5.5, 5.8, 5.1, 6.2], dtype=float32)>),
             ('sepal_width',
              <tf.Tensor: id=61, shape=(32,), dtype=float32, numpy=
              array([3.6, 3.3, 3.2, 2.3, 3.2, 3.4, 3.1, 3.2, 3.2, 3. , 3.5, 3.4, 2.8,
                     3.7, 3. , 3.8, 2.7, 2.8, 2.6, 3. , 3.1, 2.8, 2.7, 3.7, 2.7, 2.5,
                     2.2, 3. , 2.4, 2.7, 3.8, 2.2], dtype=float32)>),
             ('petal_length',
              <tf.Tensor: id=58, shape=(32,), dtype=float32, numpy=
              array([6.1, 1.4, 1.4, 3.3, 4.7, 1.5, 1.5, 1.2, 5.7, 5.2, 1.5, 1.6, 6.7,
                     1.5, 4.9, 6.7, 4.2, 6.1, 4. , 4.8, 1.5, 4.6, 5.1, 1.5, 5.3, 3.9,
                     5. , 6.1, 3.7, 4.1, 1.5, 4.5],

To simplify the model building step, create a function to repackage the features dictionary into a single array with shape: (batch_size, num_features).

This function uses the tf.stack method which takes values from a list of tensors and creates a combined tensor at the specified dimension.

In [10]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

Then use the tf.data.Dataset.map method to pack the features of each (features,label) pair into the training dataset:

In [11]:
train_dataset = train_dataset.map(pack_features_vector)

The features element of the Dataset are now arrays with shape (batch_size, num_features). Let's look at the first few examples:

In [12]:
features, labels = next(iter(train_dataset))

print(features[:5])

tf.Tensor(
[[5.5 2.6 4.4 1.2]
 [4.4 3.2 1.3 0.2]
 [4.6 3.4 1.4 0.3]
 [6.  2.7 5.1 1.6]
 [5.5 2.4 3.7 1. ]], shape=(5, 4), dtype=float32)


## Create a model using Keras

In [13]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

## Using the model

In [14]:
predictions = model(features)
predictions[:5]

<tf.Tensor: id=188, shape=(5, 3), dtype=float32, numpy=
array([[ 0.9423989 , -0.22337034,  0.70007336],
       [ 0.97970355, -0.21203095,  0.02253342],
       [ 1.0101863 , -0.22846445,  0.0384773 ],
       [ 0.9697454 , -0.24448884,  0.8523805 ],
       [ 0.9958335 , -0.19281062,  0.546317  ]], dtype=float32)>

Here, each example returns a logit for each class.

To convert these logits to a probability for each class, use the softmax function:

In [15]:
tf.nn.softmax(predictions[:5])

<tf.Tensor: id=194, shape=(5, 3), dtype=float32, numpy=
array([[0.4769892 , 0.14866935, 0.37434143],
       [0.5925322 , 0.17994851, 0.2275193 ],
       [0.5994447 , 0.17370404, 0.22685128],
       [0.4574151 , 0.1358236 , 0.40676126],
       [0.51478183, 0.15682004, 0.3283981 ]], dtype=float32)>

Taking the tf.argmax across classes gives us the predicted class index. But, the model hasn't been trained yet, so these aren't good predictions.

In [16]:
print("Prediction: {}".format(tf.argmax(predictions, axis=1)))
print("    Labels: {}".format(labels))

Prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0]
    Labels: [1 0 0 1 1 2 0 0 2 1 0 1 0 2 2 0 1 0 2 2 0 0 2 1 1 1 1 1 0 2 1 1]


## Now Train the Model

## Define the LOSS and GRADIENT Function

In [17]:
def loss(model, x, y):
  y_ = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)


l = loss(model, features, labels)
print("Loss test: {}".format(l))

Loss test: 1.1667357683181763


Use the tf.GradientTape context to calculate the gradients used to optimize our model.

In [18]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

## Create an OPTIMIZER

In [19]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

global_step = tf.train.get_or_create_global_step()

Use this to calculate a single optimization step:

In [21]:
loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(global_step.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.variables), global_step)

print("Step: {},         Loss: {}".format(global_step.numpy(),
                                          loss(model, features, labels).numpy()))

Step: 0, Initial Loss: 1.1667357683181763
Step: 1,         Loss: 1.1379096508026123


## Training loop
With all the pieces in place, the model is ready for training! A training loop feeds the dataset examples into the model to help it make better predictions.

In [22]:
## Note: Rerunning this cell uses the same model variables

# keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tfe.metrics.Mean()
  epoch_accuracy = tfe.metrics.Accuracy()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.variables),
                              global_step)

    # Track progress
    epoch_loss_avg(loss_value)  # add current batch loss
    # compare predicted label to actual label
    epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)

  # end epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  
  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 0.997, Accuracy: 70.000%
Epoch 050: Loss: 0.364, Accuracy: 93.333%
Epoch 100: Loss: 0.225, Accuracy: 96.667%
Epoch 150: Loss: 0.169, Accuracy: 98.333%
Epoch 200: Loss: 0.129, Accuracy: 99.167%


Use the trained Model to make Predictions

In [24]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

Example 0 prediction: Iris setosa (98.4%)
Example 1 prediction: Iris versicolor (90.6%)
Example 2 prediction: Iris virginica (76.0%)
